In [1]:
!pip install -U bitsandbytes

!pip install -U transformers peft accelerate
 
!pip install -U accelerate peft
 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
!pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cu126
!pip install transformers peft accelerate datasets


Looking in indexes: https://download.pytorch.org/whl/cu126
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 44.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.12.0 re

In [3]:
!pip install safetensors


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer , BitsAndBytesConfig
import torch

torch.cuda.empty_cache()

model_name = "Qwen/Qwen2.5-1.5B-Instruct"
#configuration for quantizing the qwen model (windows setup)
# bnb_cfg = BitsAndBytesConfig(
#     load_in_4bit=True,                # 4-bit
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",        
#     bnb_4bit_compute_dtype=torch.bfloat16 
# )

#linux quantization:
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,                
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",        
    bnb_4bit_compute_dtype=torch.bfloat16 
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # quantization_config=bnb_cfg,
    #torch_dtype=torch.float16,   
    quantization_config= bnb_cfg ,   
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

2025-12-22 13:44:05.838915: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766411046.205017      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766411046.312131      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [5]:
#simple adapter cache to fix later:
adapter_cache = {}
def get_adapter(adapter_name):
    if adapter_name in adapter_cache:
        return adapter_cache[adapter_name]
    model = load_adapter(f"./adapters/{adapter_name}")
    adapter_cache[adapter_name] = model
    if len(adapter_cache) > CACHE_LIMIT:
        evict_one()  # define later
    return model


In [6]:
#adapter from: https://huggingface.co/DreamGallery/Qwen-Qwen2.5-1.5B-Instruct-1727452927
# from peft import PeftModel

# model = PeftModel.from_pretrained(
#     base_model,
#     "DreamGallery/Qwen-Qwen2.5-1.5B-Instruct-1727452927",  # adapter path
#     trust_remote_code=True
# )


In [7]:
prompt = "Explain the difference between L1 and L2 regularization in machine learning."
inputs = tokenizer(prompt, return_tensors="pt").to(base_model.device)
with torch.no_grad():
    out = base_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))


Explain the difference between L1 and L2 regularization in machine learning. Provide an example to illustrate your explanation.
L1 regularization (also known as Lasso regression) adds a penalty term that is proportional to the absolute value of the coefficients to the loss function, while L2 regularization (also known as Ridge regression) adds a penalty term that is proportional to the square of the coefficients to the


In [8]:
#dataset of prompts:
from datasets import load_dataset

ds1 = load_dataset("fka/awesome-chatgpt-prompts")

README.md:   0%|          | 0.00/339 [00:00<?, ?B/s]

prompts.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/509 [00:00<?, ? examples/s]

In [9]:
from datasets import load_dataset

ds = load_dataset("RUC-DataLab/DataScience-Instruct-500K")

README.md: 0.00B [00:00, ?B/s]

multi_ability_agentic_training.json:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
from datasets import load_dataset

ds2 = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en")

README.md: 0.00B [00:00, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/58.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19704 [00:00<?, ? examples/s]

In [11]:
import datasets
print(datasets.__version__)

4.1.1


In [12]:
from datasets import Dataset

print(type(ds))   # Should be <class 'datasets.arrow_dataset.Dataset'>
print(type(ds1))  # Should be <class 'datasets.arrow_dataset.Dataset'>

<class 'datasets.dataset_dict.DatasetDict'>
<class 'datasets.dataset_dict.DatasetDict'>


In [13]:
import datasets
print(datasets.__version__)

4.1.1


In [14]:
def tokenize_ds(batch):
    prompts = []
    targets = []

    for messages, evaluation in zip(batch["messages"], batch["evaluation"]):
        if isinstance(messages, list):
            prompt = "\n".join([f"{m.get('role', '')}: {m.get('content', '')}" for m in messages])
        else:
            prompt = str(messages)

        prompts.append(prompt)
        targets.append(str(evaluation))

    return tokenizer(
        prompts,
        text_target=targets,
        truncation=True,
        padding="max_length",
        max_length=512,
    )

def tokenize_ds1(batch):
    prompts = []
    targets = []

    for act, prompt in zip(batch["act"], batch["prompt"]):
        inst = " ".join(act) if isinstance(act, list) else str(act)
        prompts.append(inst)
        targets.append(str(prompt))

    return tokenizer(
        prompts,
        text_target=targets,
        truncation=True,
        padding="max_length",
        max_length=512,
    )

def tokenize_ds3(batch):
    return tokenizer(
        batch["Question"],
        text_target=batch["Response"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )



train1 = ds.map(tokenize_ds, batched=True, batch_size=256)
train2 = ds1.map(tokenize_ds1, batched=True, batch_size=256)



Map:   0%|          | 0/26202 [00:00<?, ? examples/s]

Map:   0%|          | 0/509 [00:00<?, ? examples/s]

In [15]:
train3 = ds2.map(tokenize_ds3, batched=False)

Map:   0%|          | 0/19704 [00:00<?, ? examples/s]

In [16]:
from peft import LoraConfig 
lora_cfg_1 = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.1,
    task_type="CAUSAL_LM"
)
lora_cfg_2 = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.1,
    task_type="CAUSAL_LM"
)


In [17]:

print("CUDA available:", torch.cuda.is_available())
print("Current device:", torch.cuda.get_device_name(0))

CUDA available: True
Current device: Tesla T4


In [18]:
!nvidia-smi

Mon Dec 22 13:53:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   75C    P0             32W /   70W |     855MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
from peft import get_peft_model
from transformers import TrainingArguments, Trainer
torch.cuda.empty_cache()

model1 = get_peft_model(base_model, lora_cfg_1)
args1 = TrainingArguments(
    per_device_train_batch_size=1,
    num_train_epochs=1,
    learning_rate=2e-4,
    output_dir="./adapter1",
    gradient_accumulation_steps=1,
    logging_dir="./logs",         # where logs are saved
    logging_steps=5,              # print every 5 steps
    report_to="none",             # disable WandB, keep console output
    disable_tqdm=False            # show progress bar
)
small_train1 = train1["train"].select(range(100))  # use only first 100 samples for faster runtime 

trainer1 = Trainer(model=model1, args=args1, train_dataset=small_train1)
trainer1.train()
model1.save_pretrained("./adapters/adapter1")

del model1
torch.cuda.empty_cache()

model2 = get_peft_model(base_model, lora_cfg_2)
args2 = TrainingArguments(
    per_device_train_batch_size=1,
    num_train_epochs=1,
    learning_rate=2e-4,
    output_dir="./adapter2",
    gradient_accumulation_steps=1,
    logging_dir="./logs",         # where logs are saved
    logging_steps=5,              # print every 5 steps
    report_to="none",             # disable WandB, keep console output
    disable_tqdm=False            # show progress bar
)
small_train2 = train2["train"].select(range(100))
trainer2 = Trainer(model=model2, args=args2, train_dataset=small_train2)
trainer2.train()
model2.save_pretrained("./adapters/adapter2")


#adapter 3:

model3 = get_peft_model(base_model, lora_cfg_2)
args3 = TrainingArguments(
    per_device_train_batch_size=1,
    num_train_epochs=1,
    learning_rate=2e-4,
    output_dir="./adapter3",
    gradient_accumulation_steps=1,
    logging_dir="./logs",         # where logs are saved
    logging_steps=5,              # print every 5 steps
    report_to="none",             # disable WandB, keep console output
    disable_tqdm=False            # show progress bar
)
small_train3 = train3["train"].select(range(100))
trainer3 = Trainer(model=model3, args=args3, train_dataset=small_train3)
trainer3.train()
model3.save_pretrained("./adapters/adapter3")


Step,Training Loss
5,27.120900
10,25.735800
15,24.020200
20,22.680700
25,21.364700
30,18.747500
35,15.658600
40,12.794100
45,10.600200
50,8.072300


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Step,Training Loss
5,18.518700
10,17.691300
15,17.484200
20,17.715500
25,17.254000
30,16.758500
35,15.338000
40,14.715000
45,13.416700
50,12.387600


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Step,Training Loss
5,20.233100
10,19.401100
15,18.528200
20,18.191700
25,16.350500
30,16.879700
35,15.055500
40,13.201400
45,12.864700
50,11.984600


In [20]:
import os
from safetensors.torch import load_file

def get_dir_size_mb(path):
    total_size = 0
    for root, _, files in os.walk(path):
        for f in files:
            fp = os.path.join(root, f)
            if os.path.isfile(fp):
                total_size += os.path.getsize(fp)
    return total_size / (1024 * 1024)  # MB

def get_adapter_vram_mb(adapter_path, dtype_bytes=2):
    """
    dtype_bytes:
        fp16 / bf16 = 2
        fp32 = 4
    """
    adapter_file = os.path.join(adapter_path, "adapter_model.safetensors")
    state_dict = load_file(adapter_file)  # SAFE loader
    total_params = sum(t.numel() for t in state_dict.values())
    vram_mb = total_params * dtype_bytes / (1024 * 1024)
    return vram_mb

# Configuration
base_path = "/kaggle/working/adapters"   
adapter_names = ["adapter1", "adapter2", "adapter3"]

# Process each adapter
print("=" * 55)
print("ADAPTER SIZE ANALYSIS")
print("=" * 55)

for adapter_name in adapter_names:
    adapter_path = os.path.join(base_path, adapter_name)
    
    print(f"\n📦 {adapter_name}")
    print("-" * 30)
    
    if os.path.exists(adapter_path):
        # Disk size
        size_mb = get_dir_size_mb(adapter_path)
        print(f"  💾 Disk size:   {size_mb:.2f} MB")
        
        # VRAM size
        vram_cost = get_adapter_vram_mb(adapter_path, dtype_bytes=2)
        print(f" VRAM cost:   {vram_cost:.2f} MB")
    else:
        print(f" Directory not found: {adapter_path}")

print("\n" + "=" * 55)

ADAPTER SIZE ANALYSIS

📦 adapter1
------------------------------
  💾 Disk size:   2.10 MB
 VRAM cost:   1.04 MB

📦 adapter2
------------------------------
  💾 Disk size:   2.10 MB
 VRAM cost:   1.04 MB

📦 adapter3
------------------------------
  💾 Disk size:   2.10 MB
 VRAM cost:   1.04 MB



In [21]:
from transformers import  AutoTokenizer
from peft import PeftModel

model_name = "Qwen/Qwen2.5-1.5B-Instruct"
adapter_path = "./adapters/adapter1"

tokenizer = AutoTokenizer.from_pretrained(model_name)


model = PeftModel.from_pretrained(base_model, adapter_path)
model.eval()

text ="""### Instruction:
explain the difference between L1 and L2 regularization in machine learning.

### Input:

### Response:"""
 
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=150, temperature=0.7, top_p=0.9)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded)

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Explain the difference between L1 and L2 regularization in machine learning. L1 and L2 are two types of regularization techniques used to prevent overfitting in machine learning models.

L1 regularization, also known as Lasso regression, adds a penalty term to the loss function that is proportional to the absolute value (or length) of the model's parameters. This makes the parameter weights smaller but can lead to some features being set to zero entirely which may simplify the model and reduce its complexity and improve interpretability. It could help create sparse solutions by eliminating insignificant parameters.

In contrast, L2 regularization, commonly referred to as Ridge Regression, has a penalization factor that is proportional to the square of the magnitude of each parameter. The effect is that it reduces the size of all parameters equally. This method can result in


In [22]:
!git clone https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2


Cloning into 'all-MiniLM-L6-v2'...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


remote: Enumerating objects: 112, done.
remote: Total 112 (delta 0), reused 0 (delta 0), pack-reused 112 (from 1)
Receiving objects: 100% (112/112), 358.47 KiB | 9.69 MiB/s, done.
Resolving deltas: 100% (44/44), done.
Filtering content: 100% (15/15), 930.41 MiB | 216.58 MiB/s, done.


In [23]:

from sentence_transformers import SentenceTransformer , util

model = SentenceTransformer("./all-MiniLM-L6-v2")
adapter_tasks = [
    "role-playing instruction following across many creative and utility roles",
    "data science and machine learning problem solving with step-by-step reasoning",
    "medical diagnosis and clinical reasoning with detailed chain-of-thought explanations",
    "summarization of long articles"
]
adapter_embs = model.encode(adapter_tasks)

print(adapter_embs)
def select_adapter(user_prompt):
    prompt_emb = model.encode([user_prompt], normalize_embeddings=True)[0]
    sims = util.cos_sim(prompt_emb, adapter_embs)[0]
    top = sims.argmax().item()
    best_score = sims[top].item()
    return adapter_tasks[top], best_score
task, score = select_adapter("Can you summarize this text?")
print(task, score)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[ 0.0964161  -0.03359805  0.06819045 ...  0.06474049 -0.00220003
   0.00758782]
 [-0.09146075  0.05695623  0.0701498  ...  0.0148928  -0.00308064
   0.03536719]
 [ 0.05057738  0.04845603  0.04390772 ...  0.14514221  0.04068506
   0.01985919]
 [ 0.02673401  0.09173454  0.00614705 ...  0.05114826  0.00423551
   0.02519144]]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

summarization of long articles 0.45014241337776184


In [24]:
prompt = "Explain how blood pressure works in the human body."
adapter, scores = select_adapter(prompt)
print(adapter)   
print(scores)    


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

medical diagnosis and clinical reasoning with detailed chain-of-thought explanations
0.12989044189453125


In [25]:
# import numpy as np
# import random

# class AdapterEnv:

#     def __init__(self, adapters, gpus):
     

#         self.adapters = adapters
#         self.gpus = gpus
#         self.queue_length = 0
#         self.avg_latency = 0
#         self.throughput = 0
#         self.baseline_inference_time = 1.0  # normalized baseline

#         self.current_query = None
#         self.current_step = 0

#     # STATE
#     def _get_state(self):
#         """Return the full state vector."""

#         # Adapter characteristics
#         adapter_vec = []
#         for a in self.adapters.values():
#             adapter_vec.extend([
#                 a["size_mb"],
#                 a["mem_required"],
#                 a["load_time"],
#                 a["latency"],
#                 a["domain_id"],
#                 1 if a["loaded"] else 0,
#                 a["which_gpu"] if a["loaded"] else -1
#             ])

#         # GPU characteristics
#         gpu_vec = []
#         for g in self.gpus.values():
#             gpu_vec.extend([
#                 g["free_vram"],
#                 g["used_vram"],
#                 len(g["loaded_adapters"]),
#                 g["eviction_time"]
#             ])

#         # System context
#         sys_vec = [
#             self.queue_length,
#             self.avg_latency,
#             self.throughput
#         ]

#         return np.array(adapter_vec + gpu_vec + sys_vec, dtype=np.float32)


#     # ACTION SPACE

#     def action_space(self):
#         """
#         Actions are encoded like this:
#         type 0 → selection (pick adapter id or  -1 = no-op)
#         type 1 → placement (load / evict / reuse / swap)
#         type 2 → routing (send query to GPU id)
#         """

#         selection_actions = list(self.adapters.keys()) + [-1]  # -1 = do nothing
#         placement_actions = ["load", "evict", "reuse", "swap"]
#         routing_actions = list(self.gpus.keys())

#         return {
#             "selection": selection_actions,
#             "placement": placement_actions,
#             "routing": routing_actions
#         }

#     def sample_action(self):
#         """Randomly choose valid actions."""
#         return {
#             "selection": random.choice(self.action_space()["selection"]),
#             "placement": random.choice(self.action_space()["placement"]),
#             "routing": random.choice(self.action_space()["routing"])
#         }

#     # STEP 
#     def step(self, action, user_query):
#         """
#         user_query contains:
#             - domain_id
#             - estimated_load
#         """

#         self.current_query = user_query
#         selected_adapter = action["selection"]
#         placement = action["placement"]
#         chosen_gpu = action["routing"]

#         reward = 0
#         cost = 0

#         # SELECTION COST
#         if selected_adapter != -1:
#             correct = (self.adapters[selected_adapter]["domain_id"] ==
#                        user_query["domain_id"])
#             reward += 1 if correct else -1

#         # PLACEMENT COST (LOAD / EVICT / REUSE / SWAP)
  
#         if placement == "load":
#             a = self.adapters[selected_adapter]
#             g = self.gpus[chosen_gpu]
#             # simulate load time and VRAM use
#             cost += a["load_time"]
#             g["free_vram"] -= a["mem_required"]
#             g["used_vram"] += a["mem_required"]
#             a["loaded"] = True
#             a["which_gpu"] = chosen_gpu
#             g["loaded_adapters"].append(selected_adapter)

#         elif placement == "evict":
#             a = self.adapters[selected_adapter]
#             g = self.gpus[a["which_gpu"]]
#             # eviction time
#             cost += g["eviction_time"]
#             g["free_vram"] += a["mem_required"]
#             g["used_vram"] -= a["mem_required"]
#             a["loaded"] = False
#             a["which_gpu"] = -1
#             if selected_adapter in g["loaded_adapters"]:
#                 g["loaded_adapters"].remove(selected_adapter)

#         elif placement == "swap":
#             cost += 2  # arbitrary extra penalty

        
#         # ROUTING COST (INFERENCE)
#         if selected_adapter != -1:
#             adapter_latency = self.adapters[selected_adapter]["latency"]
#         else:
#             adapter_latency = 2  # penalize doing nothing

#         inference_time = adapter_latency
#         cost += inference_time / self.baseline_inference_time

#         # GPU balancing reward
#         gpu_loads = [g["used_vram"] for g in self.gpus.values()]
#         balance = np.std(gpu_loads)
#         reward -= balance  # penalize imbalance

#         # Final reward = positive selection score - costs
#         final_reward = reward - cost

#         # update system context
#         self.avg_latency = 0.9 * self.avg_latency + 0.1 * inference_time
#         self.queue_length = max(0, self.queue_length - 1)

#         # next state
#         next_state = self._get_state()
#         done = False

#         return next_state, final_reward, done, {}

#     # ----------------------------------------------------------------
#     # RESET
#     # ----------------------------------------------------------------
#     def reset(self):
#         self.queue_length = 0
#         self.avg_latency = 0
#         self.throughput = 0
#         for a in self.adapters.values():
#             a["loaded"] = False
#             a["which_gpu"] = -1
#         for g in self.gpus.values():
#             g["loaded_adapters"].clear()
#             g["used_vram"] = 0
#             g["free_vram"] = g["total_vram"]
#         return self._get_state()


In [26]:
import pandas as pd

df = pd.read_csv('/kaggle/input/google-cluster-sample/borg_traces_data.csv')
df.head()


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,Unnamed: 0,time,instance_events_type,collection_id,scheduling_class,collection_type,priority,alloc_collection_id,instance_index,machine_id,...,assigned_memory,page_cache_memory,cycles_per_instruction,memory_accesses_per_instruction,sample_rate,cpu_usage_distribution,tail_cpu_usage_distribution,cluster,event,failed
0,0,0,2,94591244395,3,1,200,0,144,168846390496,...,0.014435,0.000415,NaN,NaN,1.0,[0.00314331 0.00381088 0.00401306 0.00415039 0...,[0.00535583 0.00541687 0.00548553 0.00554657 0...,7,FAIL,1
1,1,2517305308183,2,260697606809,2,0,360,221495397286,335,85515092,...,0.000000,0.000000,NaN,NaN,1.0,[1.23977661e-05 1.23977661e-05 1.23977661e-05 ...,[1.23977661e-05 1.23977661e-05 1.23977661e-05 ...,7,FAIL,1
2,2,195684022913,6,276227177776,2,0,103,0,376,169321752432,...,0.010422,0.000235,0.939919,0.001318,1.0,[0.01344299 0.01809692 0.0201416 0.02246094 0...,[0.02902222 0.02929688 0.0295105 0.0296936 0...,7,SCHEDULE,0
3,3,0,2,10507389885,3,0,200,0,1977,178294817221,...,0.041626,0.000225,1.359102,0.007643,1.0,[0.03704834 0.04125977 0.04290771 0.04425049 0...,[0.05535889 0.05584717 0.05633545 0.05718994 0...,8,FAIL,1
4,4,1810627494172,3,25911621841,2,0,0,0,3907,231364893292,...,0.000272,0.000010,NaN,NaN,1.0,[0. 0. 0. 0. 0...,[0.00041485 0.00041485 0.00041485 0.00041485 0...,2,FINISH,0


In [27]:
from typing import Tuple
import gymnasium as gym
import numpy as np
from stable_baselines3 import DQN
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

MAX_GPU_VRAM_MB = 2    
MAX_DISK_MB = 100          # max adapter size 

class AdapterEnv(gym.Env):
   
    
    metadata = {"render_modes": []}

    def __init__(self, df: pd.DataFrame, num_adapters=3, num_gpus=2): #change nums here
        super().__init__()
        self.df = df
        self.num_adapters = num_adapters
        self.num_gpus = num_gpus
        self.current_idx = 0
        self.vram_capacity = 1.0
        self.baseline_latency = 0.5
        self.max_throughput = 1.0
        adapter_vram_mb = np.array([1.04] * num_adapters, dtype=np.float32)
        adapter_disk_mb = np.array([6.37] * num_adapters, dtype=np.float32
                                  )
        # GPU and adapter state
        self.adapter_loaded = np.zeros((num_gpus, num_adapters), dtype=np.float32)
        self.free_vram = np.ones(num_gpus, dtype=np.float32)
        self.queue_length = 0.0
        self.avg_latency = 0.0
        self.throughput = 0.0
        self.adapter_vram = adapter_vram_mb / MAX_GPU_VRAM_MB
        self.adapter_load_cost = adapter_disk_mb / MAX_DISK_MB
        self.adapter_last_used = np.zeros(num_adapters, dtype=np.int32)
        # Actions  
        self.num_actions = 5 * self.num_adapters
        self.action_space = gym.spaces.Discrete(self.num_actions)

        # Observation
        self.adapter_feat_len = 4
        self.gpu_feat_len = 2
        obs_size = (num_adapters * self.adapter_feat_len) + (num_gpus * self.gpu_feat_len) + 3
        self.observation_space = gym.spaces.Box(low=0.0, high=1.0, shape=(obs_size,), dtype=np.float32)

    def _get_obs(self):
        # simplified: adapter + GPU + system stats
        row = self.df.iloc[self.current_idx % len(self.df)]
        adapter_stats = []
        for a in range(self.num_adapters):
            loaded = float(self.adapter_loaded[:, a].any())
            vram_cost = self.adapter_vram[a]   # normalized
            load_cost = self.adapter_load_cost[a]
            reuse_age = min(
                (self.current_idx - self.adapter_last_used[a]) / 1000.0,1.0)
            adapter_stats.extend([loaded, vram_cost, load_cost, reuse_age])
        adapter_stats = np.array(adapter_stats, dtype=np.float32)

        gpu_stats = []
        for gpu in range(self.num_gpus):
            gpu_stats.extend([self.free_vram[gpu], np.sum(self.adapter_loaded[gpu]) ])
        gpu_stats = np.array(gpu_stats, dtype=np.float32)
        system_stats = np.array([self.queue_length, self.avg_latency, self.throughput], dtype=np.float32)
        return np.concatenate([adapter_stats, gpu_stats, system_stats]).astype(np.float32)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.adapter_loaded[:] = 0.0
        self.free_vram[:] = 1.0
        #self.queue_length = 0.5
        self.avg_latency = 0.5
        self.throughput = 0.5
        self.current_idx = 0
        return self._get_obs(), {}
        
    def step(self, action):
        # decode action type
        action_type = action // (self.num_adapters * self.num_gpus)
        rem = action % (self.num_adapters * self.num_gpus)
        adapter_id = rem // self.num_gpus
        gpu_id = rem % self.num_gpus #  send to most free GPU
        row = self.df.iloc[self.current_idx % len(self.df)]
        # ---------------- Reward shaping ---------------- #

        # 1. Adapter correctness  
        # correct_adapter = (adapter_id % 2 == row.get('collection_type', 0))
        # r_domain = 1.0 if correct_adapter else -1.0
        #   Execution reward (penalize just selecting)
        r_execution = 0.0
        if action_type == 0:  # Just selecting gets ZERO
            r_execution = 0.0
        elif action_type in [1, 3, 4]:  # load/reuse/send gets reward
            r_execution = 1.0 # if correct_adapter else -1.0

        # 2. Latency penalty
        r_latency = - (self.avg_latency / self.baseline_latency)

        # 3. Throughput reward
        r_throughput = self.throughput / self.max_throughput

        # 4. VRAM safety
        vram_used = 1.0 - self.free_vram[gpu_id]
        if vram_used > self.vram_capacity:
            r_vram = -1.0
        else:
            r_vram = 0.1 * (1 - vram_used / self.vram_capacity)

        # 5. Eviction & swap penalties
        r_evict = 0.2 if (action_type == 2 and self.free_vram[gpu_id] < np.min(self.adapter_vram)) else -0.1 if action_type == 2 else 0.0
        needs_swap = (action_type == 1 and self.adapter_loaded[gpu_id].any()and self.free_vram[gpu_id] < self.adapter_vram[adapter_id])
        r_swap = -0.6 if needs_swap else 0.0

        # 6. GPU balancing
        gpu_vram_usages = 1.0 - self.free_vram
        r_balance = -np.std(gpu_vram_usages)
        if action_type in {1, 3, 4}:  # load, reuse, send
            self.adapter_last_used[adapter_id] = self.current_idx
        # Final reward
        reward = (
            # 2.0 * r_domain +   
            0.05 * r_execution +  # ← Smaller components
            0.02 * r_latency +
            0.01 * r_throughput +
            0.01 * r_balance +
            0.001 * (r_vram + r_evict + r_swap)   
        )
                 

        # ----- Action effects ----- #
        # if action_type == 0:  # select
        #     reward -= 0.02
        #    # reward += 1.0 if correct_adapter else -1.0
        #     pass
        if action_type == 1:  # load
            vram_cost = self.adapter_vram[adapter_id]
        
            if self.free_vram[gpu_id] >= vram_cost:
                if self.adapter_loaded[gpu_id, adapter_id] == 0.0:
                    self.adapter_loaded[gpu_id, adapter_id] = 1.0
                    self.free_vram[gpu_id] -= vram_cost
                    reward -= self.adapter_load_cost[adapter_id]
                else:
                    reward -= 1  # already loaded
            else:
                reward -= 1.0  # illegal load (OOM)
        
        elif action_type == 2:  # evict
            if self.adapter_loaded[gpu_id, adapter_id] == 1.0:
                self.adapter_loaded[gpu_id, adapter_id] = 0.0
                self.free_vram[gpu_id] += self.adapter_vram[adapter_id]
            else:
                reward -= 1  # useless eviction
        
        elif action_type == 3:
            if self.adapter_loaded[gpu_id, adapter_id] == 1.0 :
                reward += 1
            else:
                reward -= 1

        
        elif action_type == 4:  # send
            if self.adapter_loaded[gpu_id, adapter_id] == 1.0:
                reward += 1.0 #if correct_adapter else -1.0
    
        # update system stats
        # self.queue_length = np.clip(self.queue_length, 0, 1)
        # self.avg_latency = self.queue_length
        # self.throughput = 1.0 - self.avg_latency
 
        # move to next job
        self.current_idx += 1
        terminated = self.current_idx >= 200
        truncated = False
         
        return self._get_obs(), float(reward), terminated, truncated,  {}

    def render(self):
        pass


# Make sure the environment works properly
env = AdapterEnv(df, num_adapters=3, num_gpus=2)
check_env(env, warn=True)
# ------------------ Training ------------------ #
model = PPO(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=256,
    ent_coef=0.02,           # More exploration
    vf_coef=0.5,             # Focus on value function
    gamma=0.99
)

# Train for a reasonable number of timesteps
print("Adapters:", env.num_adapters, "GPUs:", env.num_gpus, "Actions:", env.action_space.n)
#action, _states = model.predict(obs, deterministic=False)
model.learn(total_timesteps=100_000)

# Save the trained model
model.save("adapter_ppo")
print("Training complete.")

# ------------------ Inference ------------------ #
obs, _ = env.reset()
for i in range(50):  # test 20 steps
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    print(f"Step {i}: Action {action}, Reward {reward}")
    if terminated:
        obs, _ = env.reset()
    

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Adapters: 3 GPUs: 2 Actions: 15
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 200      |
|    ep_rew_mean     | -109     |
| time/              |          |
|    fps             | 628      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | -102        |
| time/                   |             |
|    fps                  | 578         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.017619506 |
|    clip_fraction        | 0.23        |
|    clip_range           | 0

In [28]:
import gymnasium as gym
import numpy as np

MAX_GPU_VRAM_MB = 2
MAX_DISK_MB = 100

class AdapterEnv(gym.Env):

    def __init__(self, num_adapters=3, num_gpus=2):
        super().__init__()

        self.num_adapters = num_adapters
        self.num_gpus = num_gpus

        self.adapter_vram = np.array([1.04] * num_adapters) / MAX_GPU_VRAM_MB
        self.adapter_load_cost = np.array([6.37] * num_adapters) / MAX_DISK_MB

        # 4 actions × adapter × gpu
        self.num_actions = 4 * num_adapters * num_gpus
        self.action_space = gym.spaces.Discrete(self.num_actions)

        self.reset()

    def reset(self, seed=None, options=None):
        self.adapter_loaded = np.zeros((self.num_gpus, self.num_adapters), dtype=int)
        self.free_vram = np.ones(self.num_gpus)

        self.queue_length = np.random.randint(5, 15)   ### NEW
        self.step_count = 0

        return self._get_state(), {}

    def _get_state(self):
        return (
            tuple(self.adapter_loaded.flatten()),
            tuple(np.round(self.free_vram, 1)),
            self.queue_length                               ### NEW
        )

    def step(self, action):
        action_type = action // (self.num_adapters * self.num_gpus)
        rem = action % (self.num_adapters * self.num_gpus)
        adapter_id = rem // self.num_gpus
        gpu_id = rem % self.num_gpus

        reward = 0.0

        # ---- LOAD ----
        if action_type == 0:
            cost = self.adapter_vram[adapter_id]
            if self.free_vram[gpu_id] >= cost and self.adapter_loaded[gpu_id, adapter_id] == 0:
                self.adapter_loaded[gpu_id, adapter_id] = 1
                self.free_vram[gpu_id] -= cost
                reward = +0.5
            else:
                reward = -0.5

        # ---- EVICT ----
        elif action_type == 1:
            if self.adapter_loaded[gpu_id, adapter_id] == 1:
                self.adapter_loaded[gpu_id, adapter_id] = 0
                self.free_vram[gpu_id] += self.adapter_vram[adapter_id]
                reward = +0.2
            else:
                reward = -0.2

        # ---- REUSE ----
        elif action_type == 2:
            reward = +0.3 if self.adapter_loaded[gpu_id, adapter_id] else -0.3

        # ---- SEND (ACTUAL WORK) ----
        elif action_type == 3:
            if self.adapter_loaded[gpu_id, adapter_id] and self.queue_length > 0:
                self.queue_length -= 1                    ### NEW
                reward = +1.0                              ### REAL PROGRESS
            else:
                reward = -1.0

        # ---- Arrival process ----
        arrivals = np.random.poisson(0.3)                 ### NEW
        self.queue_length += arrivals
        self.queue_length = max(self.queue_length, 0)

        self.step_count += 1

        # Episode ends when work is done OR timeout
        terminated = self.queue_length == 0 or self.step_count >= 200  ### NEW

        return self._get_state(), reward, terminated, False, {}


In [29]:
from collections import defaultdict
import random
import numpy as np

env = AdapterEnv(num_adapters=3, num_gpus=2)

# Q-table
Q = defaultdict(lambda: np.zeros(env.action_space.n))

alpha = 0.1
gamma = 0.95
epsilon = 0.2
episodes = 5000

for ep in range(episodes):
    state, _ = env.reset()
    done = False

    while not done:
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(Q[state])

        next_state, reward, done, _, _ = env.step(action)

        # Q-learning update
        Q[state][action] += alpha * (
            reward + gamma * np.max(Q[next_state]) - Q[state][action]
        )

        state = next_state

    if ep % 500 == 0:
        print(f"Episode {ep}")


Episode 0
Episode 500
Episode 1000
Episode 1500
Episode 2000
Episode 2500
Episode 3000
Episode 3500
Episode 4000
Episode 4500


In [30]:
print("\n--- Q-table inspection ---")

sample_state = next(iter(Q))
print("Sample state:")
print(sample_state)

print("\nQ-values:")
for i, v in enumerate(Q[sample_state]):
    print(f"Action {i:2d}: {v:.3f}")



--- Q-table inspection ---
Sample state:
((0, 0, 0, 0, 0, 0), (1.0, 1.0), 9)

Q-values:
Action  0: 14.336
Action  1: 3.363
Action  2: 3.683
Action  3: 6.356
Action  4: 4.520
Action  5: 5.557
Action  6: 2.457
Action  7: 2.376
Action  8: 4.996
Action  9: 2.076
Action 10: 8.228
Action 11: 3.068
Action 12: 5.507
Action 13: 8.894
Action 14: 5.826
Action 15: 6.707
Action 16: 5.424
Action 17: 3.092
Action 18: 5.144
Action 19: 8.526
Action 20: 4.395
Action 21: 6.850
Action 22: 4.927
Action 23: 7.170
